In [1]:
app_py_content = """
from flask import Flask, request, jsonify
import os
import string
import re
import torch
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from transformers import BertModel, BertTokenizer, BartForConditionalGeneration, BartTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from preprocess import preprocess_text

app = Flask(__name__)

@app.route('/query', methods=['POST'])
def handle_query():
    query_text = request.json.get('query')
    if not query_text:
        return jsonify({'error': 'Query text is required.'}), 400

    # Preprocess and tokenize the query
    preprocessed_query = preprocess_text(query_text)
    encoded_query = tokenizer(preprocessed_query, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

    # Process the query with the fine-tuned BERT model to obtain query embedding
    with torch.no_grad():
        outputs = model(**encoded_query)
        query_embedding = outputs.pooler_output.cpu().detach().numpy()  # Get the query embedding

    # Compute cosine similarity between query and document embeddings
    similarity_scores = cosine_similarity(query_embedding, document_embeddings)

    # Rank and retrieve top similar/relevant documents
    top_k = 5  # Number of top documents to retrieve
    top_documents_indices = similarity_scores.argsort()[0][-top_k:][::-1]

    # Generate summaries of top similar documents
    similar_documents = []
    for idx in top_documents_indices:
        document = texts[idx]

        # Generate summary using BART
        inputs = bart_tokenizer([document], max_length=1024, return_tensors='pt', truncation=True)
        summary_ids = bart_model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)
        summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Add similarity score and summary to the list
        similarity_score = similarity_scores[0][idx]
        similar_documents.append({'similarity_score': similarity_score, 'summary': summary})

    return jsonify({'similar_documents': similar_documents})

if __name__ == '__main__':
    app.run(debug=True)
"""

with open('app.py', 'w') as file:
    file.write(app_py_content)

In [2]:
preprocess_py_content = """
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\\d+', 'NUM', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in tokens]
    return stemmed_words
"""

with open('preprocess.py', 'w') as file:
    file.write(preprocess_py_content)

In [3]:
python app.py

SyntaxError: invalid syntax (945115591.py, line 1)